In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Imports

In [1]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import seaborn as sns
from xgboost import XGBRegressor

input_path = Path('/kaggle/input/tabular-playground-series-jan-2021/')

In [1]:
train = pd.read_csv(input_path / 'train.csv', index_col='id')
display(train.head())

In [1]:
test = pd.read_csv(input_path / 'test.csv', index_col='id')
display(test.head())

In [1]:
submission = pd.read_csv(input_path / 'sample_submission.csv', index_col='id')
display(submission.head())

# EDA

In [1]:
desc=train.apply(pd.DataFrame.describe)
desc

CHECKING NULL VALUE

In [1]:
features = [f'cont{x}'for x in range(1,15)]
data= train[features]
train.isnull().sum()

## Target Distribution

In [1]:
#setting up the platform
sns.set_style("white")
sns.set_color_codes(palette='deep')
f, ax = plt.subplots(figsize=(8, 7))

sns.distplot(train['target'], color="g")
ax.xaxis.grid(False)
ax.set(ylabel="Values")
ax.set(xlabel="Target")
ax.set(title="Target distribution")
sns.despine(trim=True, left=True)
plt.show()

**PLOTING**

In [1]:
i = 1
plt.figure()
fig, ax = plt.subplots(5, 3,figsize=(14, 24))
for feature in features:
    plt.subplot(5, 3,i)
    sns.distplot(train[feature],color="blue", kde=True,bins=120, label='train')
    sns.distplot(test[feature],color="red", kde=True,bins=120, label='test')
    plt.xlabel(feature, fontsize=9); plt.legend()
    i += 1
plt.show()

Scatter Plot for Better Understanding

In [1]:
i = 1
plt.figure()
fig, ax = plt.subplots(5, 3,figsize=(14, 24))
for feature in features:
    plt.subplot(5, 3,i)
    sns.scatterplot(x=train[feature],y = train['target'],color="blue", label='train')
    plt.xlabel(feature, fontsize=9); plt.legend()
    i += 1
plt.show()

Finding target With 0 value

In [1]:
train[train['target'] == 0]

In [1]:
for col in train.columns[:-1]:
    plt.boxplot([train[feature], test[feature]], labels=['train', 'test'])
    plt.title(col)
    plt.legend()
    plt.show()
    sns.set()

Removing Outliner with IQR

In [1]:
def replace_outliers(data):
    for col in data.columns:
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1
        median_ = data[col].median()
        data.loc[((data[col] < Q1 - 1.5*IQR) | (data[col] > Q3 + 1.5*IQR)), col] = median_
    return data

In [1]:
train = replace_outliers(train)

In [1]:
corr = train.corr()
plt.subplots(figsize=(15,12))
sns.heatmap(corr, vmax=0.9, cmap="YlGnBu", square=True)

In [1]:
corr_feature = ['cont6','cont7','cont8','cont9','cont10','cont11','cont12','cont13']
corr_in = train[corr_feature]

Strong Correlation b/w corr_feature

In [1]:
corr = corr_in.corr()
plt.subplots(figsize=(15,12))
sns.heatmap(corr, vmax=0.9, cmap="YlGnBu", square=True)

In [1]:
train_data = ( train - train.mean())/train.std()
test_data = ( test - test.mean())/test.std()

Dropping feild with target value 0

In [1]:
train = train.drop(284103)

In [1]:
target = train.pop('target')
X_train, X_test, y_train, y_test = train_test_split(train, target, train_size=0.60)

using XGBRegressor

In [1]:
#parameters used from another kaggle notebook 
model = XGBRegressor(n_estimators=500, learning_rate=0.05, n_jobs=4)
model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_test, y_test)],
             verbose=False)

In [1]:
y_pred = model.predict(X_test)
score  = mean_squared_error(y_test, y_pred, squared=False)
print(score)

In [1]:
submission['target'] = model.predict(test)
submission.to_csv('xgb_regressor.csv')